In [ ]:
import numpy as np
import pandas as pd
from resources import *

In [ ]:
R_file = 'data\small_movies_R.csv'
W_file = 'data\small_movies_W.csv'
X_file = 'data\small_movies_X.csv'
Y_file = 'data\small_movies_Y.csv'
b_file = 'data\small_movies_b.csv'


In [ ]:
R = import_movies_data(R_file)
Y = import_movies_data(Y_file)
movie_list = import_movie_list()
movies = movie_list['title']
movies = np.array(movies).reshape(-1,1)

num_movies = R.shape[0]

In [ ]:
new_user = np.zeros((Y.shape[0],1))
# say = "continue"
# while say.lower() != "exit" :
#     a = int(input("Enter the Movie ID :  "))
#     while a > Y.shape[0] or a < 0 :
#         a = int(input("Enter the Movie ID which exist int real : "))
    
#     b = float(input(f"Enter the rating you give to {movies[a]} afterwatching it : "))
#     while b > 5 or b < 0: 
#         b = float(input(f"Enter a rating between 0 to 5 ,  it can be decimal too ! : "))

#     new_user[a] = b 
#     say = input("type 'exit' to stop writing the movies otherwise write 'con' to continue typing movies")
new_user = np.zeros((num_movies,1))
new_user[62] = 1
new_user[2716] = 5
new_user[3618] = 1
new_user[1398] = 5
new_user[3083] = 5
new_user[2112] = 3    
new_user[2755] = 4
new_user[4170] = 4
new_user[378] = 5
new_user[3773] = 3
new_user[4415] = 4
new_user[3009] = 1
new_user[1088] = 1
new_user[3556] = 5

my_rated = [i for i in range(num_movies) if new_user[i] > 0 ]
for i in range(num_movies):
  if(new_user[i] > 0):
    print(f"User rated {float(new_user[i])} to movie : {movies[i]}")


In [ ]:
# movies[new_user != 0] , new_user[new_user != 0]
Y = np.c_[new_user , Y]
R = np.c_[(new_user != 0).astype(int), R]
Y.shape , R.shape

In [ ]:
Y_norm = mean_normalize_data2(Y , R)
num_users = R.shape[1]
num_genre = 27 

In [ ]:
W = np.ones((Y.shape[1],num_genre))
X = np.ones((Y.shape[0],num_genre))
b = np.zeros((1,Y.shape[1]))

In [ ]:
def dJ_dw(W,X,R,Y,b,lambda__):
    ans = np.zeros((W.shape[0],W.shape[1]))
    for i in range(num_users):
        sum = np.zeros((X.shape[1],1))
        wi = np.array(W[i]).reshape(-1,1)
        for j in range(num_movies):
            if R[j][i] == 1 :
                xj = np.array(X[j]).reshape(-1,1)
                sum = sum + (wi.T.dot(xj) + b[0][i] - Y[j][i])*xj
        ans[i] = lambda__*W[i] + sum.T
    return ans  

In [ ]:
def dJ_dX(W,X,R,Y,b,lambda__):
    ans = np.zeros((X.shape[0],X.shape[1]))
    for i in range(num_movies):
        sum = np.zeros((X.shape[1],1))
        xi = np.array(X[i]).reshape(-1,1)
        for j in range(num_users):
            if R[i][j] == 1  :
                wj = np.array(W[j]).reshape(-1,1)
                sum = sum + (wj.T.dot(xi) + b[0][j] - Y[i][j])*wj 
        ans[i] = lambda__*X[i] + sum.T
    return ans


In [ ]:
def dJ_db(W,X,R,Y,b):
    ans = np.zeros((b.shape[0],b.shape[1]))
    for i in range(num_users):
        sum = 0
        wi = np.array(W[i]).reshape(-1,1)
        for j in range(num_movies):
            if R[j][i] == 1 :
                xj = np.array(X[j]).reshape(-1,1)
                sum = sum + wi.T.dot(xj) + b[0][i] - Y[j][i]
        ans[0][i] = sum
    return ans

In [ ]:
def cost_fucntion(W,X,Y,R,b,lambda__):
    cost = 0

    for i in range(num_movies):
        for j in range(num_users):
            if R[i][j] == 1:
                wj = np.array(W[j]).reshape((-1,1))
                xi = np.array(X[i]).reshape((-1,1))
                cost = cost +  (wj.T.dot(xi) + b[0][j] - Y[i][j])**2 
                                            
    for i in range(num_movies):
        for k in range(num_genre):
            cost = cost +  (lambda__/2)*X[i][k]
    
    for j in range(num_users):
        for k in range(num_genre):
            cost = cost +  (lambda__/2)*W[j][k]

    return cost                      

In [ ]:
iterations = 10
alpha = 1e-3
lambda__ = 0
cost_old  = cost_fucntion(W,X,Y,R,b,lambda__)
for i in range(iterations):
    W = W - alpha*dJ_dw(W,X,R,Y,b,lambda__)
    X = X - alpha*dJ_dX(W,X,R,Y,b,lambda__)
    b = b - alpha*dJ_db(W,X,R,Y,b)
    cost = cost_fucntion(W,X,Y,R,b,lambda__)
    if cost < cost_old :
        print(f"Cost function value after the iteration {i} is : {cost}")
    else :
        break
    

In [ ]:
WI = np.array(W[0]).reshape(-1,1)
predicted_for_user = X.dot(WI)
WI
